# Surprise

## Surprise 패키지 소개
- [07. 행렬 분해를 이용한 잠재 요인 협업 필터링 실습.ipynb](./07.%20행렬%20분해를%20이용한%20잠재%20요인%20협업%20필터링%20실습.ipynb)은 최적화나 수행 속도 측면에서 보완이 필요함
- Surprise : 추천 시스템 구축을 위한 전용 패키지
- 설치 : `Anaconda Prompt` 관리자 권한으로 실행 - `conda install -c conda-forge scikit-surprise`
    - Microsoft Visual Studio Build Tools가 설치되어 있어야 함

### 장점
- 다양한 추천 알고리즘을 가지고 있음
    - 사용자 또는 아이템 기반 최근접 이웃 협업 필터링, SVD, SVD++, NMF 기반의 잠재 요인 협업 필터링 등
- 사이킷런의 핵심 API와 유사한 API를 가지고 있음
    - `fit()`, `predict()`, `train_test_split()`, `cross_validate()`, `GridSearchCV()` 등

In [1]:
import surprise

surprise.__version__

'1.1.1'

## Surprise를 이용한 추천 시스템 구축
- [Surprise 공식 문서](https://surprise.readthedocs.io/en/stable/)

In [2]:
from surprise import SVD, Dataset, accuracy
from surprise.model_selection import train_test_split

In [3]:
data = Dataset.load_builtin('ml-100k') # 10만 개 평점 데이터, ml-1m : 100만 개 평점 데이터

trainset, testset = train_test_split(data, test_size=.25, random_state=0)

무비렌즈에서 다운 받은 파일보다 구버전 데이터

### 주의점
- 원본 데이터를 데이터 세트로 적용해야 함 (pivot-table 상태로 줄 필요 없이 알아서 변환해줌)

### SVD로 잠재 요인 협업 필터링

In [4]:
algo = SVD()
algo.fit(trainset)

### 테스트 데이터 세트에 대해 추천 수행
- `test()` : 사용자-아이템 평점 데이터 세트 전체에 대해서 추천을 예측
- `predict()` : 개별 사용자와 영화에 대한 추천 평점 반환

In [6]:
# test()
predictions = algo.test(testset)
print('prediction type :', type(predictions), 'size :', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'> size : 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.2148091460366888, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.8942691625041586, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.882256264919378, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.4494482005099303, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1414874760522356, details={'was_impossible': False})]

#### 결과 해석
- uid : 개별 사용자 아이디
- iid : 영화(또는 아이템) 아이디
- r_ui : 실제 평점
- est : 추천 예측 평점
- details : 로그 데이터, was_impossible=True : 예측 값을 생성할 수 없음

In [7]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.2148091460366888),
 ('882', '291', 3.8942691625041586),
 ('535', '507', 3.882256264919378)]

In [8]:
# predict()
# uid, iid는 string으로 입력해야함
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.33   {'was_impossible': False}


### 성능 평가

In [9]:
accuracy.rmse(predictions)

RMSE: 0.9454


0.9454024667472628

## Surprise 주요 모듈 소개
### Dataset
- 첫 번째 칼럼을 사용자 아이디, 두 번째 칼럼을 아이템 아이디, 세 번째 칼럼을 평점으로 가정하고 데이터를 로딩
- 네 번째 칼럼부터는 로딩을 아예 하지 않음
- `Dataset.load_bulitin(name='ml-100k')` : 무비렌즈 아카이브 FTP 서버에서 데이터를 다운받음
    - ml-100k, ml-1M
    - .surprise_data에 저장
    - default : ml-100k
- `Dataset.load_from_file(file_path, reader)` : OS 파일에서 데이터 로딩
    - ',', '\t' 등으로 칼럼이 분리된 포맷의 파일 로딩
    - OS 파일명, 파일의 포맷 지정
- `Dataset.load_from_df(df, reader)` : pandas DataFrame에서 데이터 로딩
    - 사용자 아이디, 아이템 아이디, 평점 순으로 칼럼 순서가 정해져 있어야 함
    - DataFrame 객체, 파일 포맷 지정

### OS 파일 데이터를 Surprise 데이터 세트로 로딩
- 주의점 : 헤더 문자열이 있으면 안 됨 (칼럼의 이름)

In [10]:
import pandas as pd

ratings = pd.read_csv('../data/ml-latest-small/ratings.csv')

# 인덱스와 헤더를 모두 제거한 파일 생성
ratings.to_csv('../data/ml-latest-small/ratings_noh.csv', index=False, header=False)

#### Reader 클래스
- `line_format` : string, 칼럼을 순서대로 나열, 입력된 문자열을 공백으로 분리해 칼럼으로 인식
- `sep` : char, 칼럼을 분리하는 분리자, default는 '\t', pandas DataFrame에서 입력받을 경우에는 필요 없음
- `rating_scale` : tuple, optional, 최소~최대 평점, 기본값은 (1, 5)

In [11]:
from surprise import Reader

# line_format : 칼럼 구성, sep : 분리 문자, rating_scale : 최소, 최대 평점
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('../data/ml-latest-small/ratings_noh.csv', reader=reader)

In [12]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)

algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### pandas DataFrame에서 Surprise 데이터 세트로 로딩

In [13]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('../data/ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5, 5))

# 순서를 맞춰줘야함
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

## Surprise 추천 알고리즘 클래스
- `SVD` : 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD 알고리즘
    - 비용 함수 : 사용자 베이스라인(Baseline) 편향성을 감안한 평점 에측에 Regularization을 적용한 것  
        $\min \sum \left( r_{ui} - \hat r_{ui}\right)^2 + \lambda \left(b_i^2 + b_u^2 + \lVert q_i \rVert^2 + \lVert p_u \rVert ^2 \right)$  
    - 사용자 예측 Rating $\hat r_{ui}=\mu+b_u+b_i+q^T_ip_u$
- `KNNBasic` : 최근접 이웃 협업 필터링을 위한 KNN 알고리즘
- `BaselineOnly` : 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘
- 이 외에 `SVD++`, `NMF` 등이 있음
    - https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html
    

### SVD 클래스 입력 파라미터
- `n_factors` : 잠재 요인 K의 개수. 커질수록 정확도는 높아질 수 있으나 과적합 발생 가능성 높아짐. 기본값 100
- `n_epochs` : SGD 수행 반복 횟수. 기본값 20
- `biased` : 베이스라인 사용자 편향 적용 여부. 기본값 True
- 예측 성능 벤치마크 결과 : http://surpriselib.com/
    - SVD++ 알고리즘의 RMSE, MAE 성적이 가장 좋지만 상대적으로 시간이 너무 오래 걸린다는 단점이 있음 (시간에 비하여 탁월한 성적이 아님)
    - SVD, k-NN Baseline이 다음으로 좋음

## 베이스라인 평점
- 개인의 성향을 편향성 요소로써 반영하여 평점을 부과하는 것을 Baseline Rating이라고 함
- 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수 공식으로 계산됨
    - 전체 평균 평점 : 모든 사용자의 아이템에 대한 평점을 평균한 값
    - 사용자 편향 점수 = 사용자별 아이템 평점 평균 값 - 전체 평균 평점
    - 아이템 편향 점수 = 아이템별 평점 평균 값 - 전체 평균 평점
- 예시) 전체 평균 평점이 3.5, 영화 A의 평균 평점이 4.2, 사용자 ㄱ의 평균 평점이 3.0일 때
    - 3.5 + (3.0-3.5) + (4.2-3.5) = 3.5 - 0.5 + 0.7 = 3.7

## 교차 검증과 하이퍼 파라미터 튜닝

### `cross_validate()`

In [15]:
from surprise.model_selection import cross_validate

ratings = pd.read_csv('../data/ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8777  0.8789  0.8781  0.8620  0.8789  0.8751  0.0066  
MAE (testset)     0.6784  0.6706  0.6741  0.6619  0.6758  0.6722  0.0057  
Fit time          3.62    3.63    3.65    3.62    3.62    3.63    0.01    
Test time         0.12    0.12    0.12    0.12    0.12    0.12    0.00    


{'test_rmse': array([0.87766936, 0.87892256, 0.87809135, 0.86197994, 0.87887129]),
 'test_mae': array([0.67836278, 0.67062044, 0.67414849, 0.66194324, 0.67581822]),
 'fit_time': (3.6193807125091553,
  3.6349902153015137,
  3.6450116634368896,
  3.617988109588623,
  3.6200146675109863),
 'test_time': (0.12092852592468262,
  0.12003779411315918,
  0.1199643611907959,
  0.11902904510498047,
  0.11897897720336914)}

### `GridSearchCV()`

In [16]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터
param_grid = {'n_epochs':[20, 40, 60], 'n_factors':[50, 100, 200]}

# CV 폴드=3개
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# RMSE 기준 최고 성능, 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8766110331701152
{'n_epochs': 20, 'n_factors': 50}


## Surprise를 이용한 개인화 영화 추천 시스템 구축

In [17]:
# train test split하지 않은 데이터 셋은 학습할 수 없음
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'global_mean'

In [18]:
# 전체 데이터를 학습 데이터 셋으로
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data_folds = DatasetAutoFolds(ratings_file='../data/ml-latest-small/ratings_noh.csv', reader=reader)

trainset = data_folds.build_full_trainset()

In [19]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

### 특정 유저의 특정 영화 예상 평점

In [20]:
movies = pd.read_csv('../data/ml-latest-small/movies.csv')

movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count()==0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [21]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


### 특정 유저의 전체 영화 예상 평점

In [22]:
# 보지 않은 영화 id 리스트
def get_unseen_surprise(ratings, movies, userId):
    # userId에 해당하는 사용자가 평점을 매긴 모든 영화 리스트
    seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()
    
    # 모든 영화 id 리스트
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화 중 본 영화 제외
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수 :', len(seen_movies), '추천 대상 영화 수 :', len(unseen_movies), '전체 영화 수 :', len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수 : 46 추천 대상 영화 수 : 9696 전체 영화 수 : 9742


In [25]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    
    # predict를 반복 수행
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # [Prediction(uid='9', iid='1', est=3.69), ...]의 리스트
    # 이를 est 값으로 정렬하기 위해 새로운 함수 정의
    def sortkey_est(pred):
        return pred.est
    
    # est 기준으로 내림차순 정렬
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]
    
    # 영화의 정보, 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    
    top_movie_preds = [ (id, title, rating) for id, title, rating in
                      zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

In [26]:
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

In [27]:
print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
    print(top_movie[1], ':', top_movie[2])

##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
